### [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
#### Authors: Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, Jeffrey Dean

### Page 1
#### Introduction
- Skip-gram is an efficient method for learning vector representations
- Word2vec offers an extension to improve the method by the following
    - Subsampling frequent words => increase speed up
    - Negative sampling
- Distributed representation helps NLP by grouping similar words
- Mikolov also introduced skip-gram model
    - Not requiring dense matrix multplication
    - Single machine with 100 billion words in 1 day
- Example
    - "Madrid" - "Spain" + "France = "Paris"

### Page 2
- Subsampling frequents words => 2x ~ 10x speed up + accuracy improvement
- Simplified varaint of Noise Contrastive Estimation (NCE) => faster training + better vector representaiton for frequent words
    - In comparison to more complex hierarchical softmax
- Word representation doesn't work well with idiomatic phrases
    - So just use word vectors to represent phrases tend to work pretty well
    - Find a bunch of phrases
    - Treat the phrases as individual tokens

#### Skip Gram Model
- Objective is to find word representation that are useful for predicting the surrounding words in a sentence or a document
- Given a sequence of training words $w_1, w_2, w_3, ..., w_T$, the model is to maximize the average log probability $$\frac{1}{T}\sum_{t=1}^{T}\sum_{-c \le j \le c, j \ne 0}log (P(w_{t+j}|w_t)) $$ where $c$ is the size of the training context
    - Larger $c$ results in more training examples and thus can lead to a higher accuracy, but there's a time
- The basic skip-gram formualtion defins $P(w_{t+j}|w_t)$ using softmax function
    - $$P(w_O|w_I) = \frac{exp(v`_{w_O}^{T}v_{w_I})}{\sum_{w=1}^{W}exp(v`_w^{T}v_{w_I})}$$ where $v_w$ is the input vector representation of $w$ and $v`_w$ is the output representation
    - Thi is pretty much impractical because the computation cost of the gradient is expensive (proportional to W)

### Page 3
#### Hierarchical Softmax
- Efficient approximation of the full softmax
- Instead of evaluating W output nodes like softmax (multi-class classification), it just needs to evaluate only about $log_2(W)$ nodes
    - To achieve that, it uses a binary tree repsentation of the output layer with W nodes as its leaves, and for each node, explicityly represents the relative probabilities of its child nodes
- $P(w|w_I) = \prod_{j = 1}^{L(w) - 1} \sigma([[n(w, j + 1) = ch(n(w, j))]] \cdot v`_{n(w, j)}^{T}v_{w_I})$
    - Each word $w$ can be reached by a path from root of the tree. 
    - $n(w, j)$ = jth node on the path from root to $w$
    - $L(w)$ is the length of the path
    - $n(w, 1)$ = root and $n(w, L(w)) = w$
    - $ch(n)$ = an arbitrary fixed child of n
    - $[[x]] = 1$ if $x$ is true and $-1$ otherwise
    - $\sigma(x) = \frac{1}{1 + e^{-x}}$
- Now the gradient is proportional to $L(W_O)$
- Unlike the standard softmax formulation of skipgram which one representation of $v_w$ for each word $w$w and one representation of $v`_n$ for every innde node $n$ of the binary tree
- Using binary tree can speed up on performance, and in their work, they have utilized huffman tree to implement this idea

### Page 4
#### Negative Sampling
- Alternative to the hierachical soft is Noise Contrastive Estimation (NCE)
- NCE can approximately maximize the log probability of softmax, the skipgram model is only concerned with learning high quality vector representations, so we are free to simplify NCE as long as the vector representations retain their quality
- Negative sampling = $log \sigma(v`_{w_O}^{T}v_{w_I}) + \sum_{i=1}^{k} \mathop{\mathbb{E}_{w_i ~ P_n(w)}}[log \sigma(-v`_{w_i}^{T}v_{w_I})]$ which is used to replace every $log P(w_O|w_I)$ term in the skip gram objective
- Thus the task is to distinguish the target word $w_O$ from draws from the noise distribution $P_n(w)$ using logistic regression, where there are $k$ negative samples for each data sample

#### Subsmapling of frequent words
- In a large corpora, the most frequent words can esaily occur hundreds of millions of times. Such words are pretty useless and proivide less information value
- The vector representations of frequent words do not change significantly after training on several million examples
- To counter this imbalance, each word $w_i$ in the training set is discarded with the probability computed by the formula $P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$
    - $f(w_i)$ is the frequency of the word $w_i$
    - $t$ is a chosen threshold, typically around $10^{-5}$
- This formula aggressively subsamples words whose frequency is greater than $t$ while preserving the ranking of frequencies
- This is a heuristic but it works well in practice and it accelerates learning and significantly improves the accuracy of the learned vectors of the rare words

### Page 5
#### Empirical Result
- Evaluating hierarchical softmax, noise contrastive estimation, negative sampling, and subsampling of the training words
- Result shows that negative sampling outperforms the hierarchical softmax and slightly better than NCE
- Utilizing $10^{-5}$ subsampling of frequent words improve the training speed by a good amount and also improve a good chunk of accuracy

### Page 6
#### Learning phrases
- To learn the vector representations for phrases, first find words that appear frequently together
- phrases are formed based on the unigram and bigram counts using the following
    - $score(w_i, w_j) = \frac{count(w_iw_j) - \sigma}{count(w_i) * count(w_j)}$
        - $\sigma$ is used as a discounting coefficient and prevents too many phrases consisting of very infrequent words to be formed
    - Bigrams with score above the chosen threshold are then used as phrases
- Typically, run 2 ~ 4 passes over the training data with decresing threshold value, allow longer phrases to be formed
#### Phrase Skip-Gram Results
- Hierarchical softmax performs the best when subsampling is happening
- Large of amount of training data is crucial for the accuracy (72% to 66% drop from 33B words to 6B words)

### Page 7
#### Additive Compositionality
- Skip gram model can perform precise analogical reasnoning using simple vector arithmetics
- The word vectors are in a linear relaitonship with the inputs to the softmax nonlinearity
- As the word vectors are trained to predict the surrounding words in the sentence, the vectors can be seen as representing the distribution of the context in which a word appears
- These values are related logarithmically to the probabilties computed by the output layer, therefore the sum of two word vectors is related to the product of the two context distributions (the product works as an AND function)
- Words that are assigned high probabilities by both word vectors will have high probability, and other words will have low probability

### Page 8
#### Comparison to other word representations & Conclusion
- Skip gram is the best one vs Collobert and Weston, Turian, and Mnih and Hinton
- Breakthrough in terms of word representation
- Choosing hyper-parameter and algorithm is a task specific decision
- Skip gram is able to train on several orders of magnitude more data than the previously published models

### Extra notes
- The way to generate the vector for a word (training in general)
    - Initialize vectors with random weights
    - For each element in the vector (the size is adjustable)
    - Essentially running the softmax to get a probability of your $P(w_{t+j}|w_t)$, and that's 1 element of the vector
    - Therefore, the vector just represent a distribution of probabilities of your surrounding words
    - From the steps above, that's just 1 training for a word vector
    - Now, run through the corpiuis to train the model (no gradient descent involved) for 1 iteration, and this is an iterative algorithm
    - AKA skip gram
- Optimization on regular standard softmax
    - Hierachical softmax uses a binary tree to reduce time/space complexity, but also it's a good approximation
- Optimization on skip gram
    - Using negative sampling to replace the $logP(W_O|W_t)$ term to approximate the probability
- Optimization on general accuracy
    - Using subsampling to improve accuracy + training speed
        - The word frequency in the denominator is similar to TFIDF